# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [36]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd

import string
import operator

# download necessary NLTK data
import nltk
nltk.download(['punkt', 'wordnet','stopwords','averaged_perceptron_tagger'])

import re
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.base import BaseEstimator, TransformerMixin

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///Messages.db')
df = pd.read_sql("SELECT * FROM Messages", engine)
df.head()



,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

In [4]:
#change the response 2 in related to 1, since I only need to know that it happened.
Y['related']=Y['related'].apply(lambda x: 1 if x == 2 else x)


In [5]:
#Check the value counts for each column of Y
for c in Y.columns:
    print(Y[c].value_counts())

1    20094
0     6122
Name: related, dtype: int64
0    21742
1     4474
Name: request, dtype: int64
0    26098
1      118
Name: offer, dtype: int64
0    15356
1    10860
Name: aid_related, dtype: int64
0    24132
1     2084
Name: medical_help, dtype: int64
0    24903
1     1313
Name: medical_products, dtype: int64
0    25492
1      724
Name: search_and_rescue, dtype: int64
0    25745
1      471
Name: security, dtype: int64
0    25356
1      860
Name: military, dtype: int64
0    26216
Name: child_alone, dtype: int64
0    24544
1     1672
Name: water, dtype: int64
0    23293
1     2923
Name: food, dtype: int64
0    23902
1     2314
Name: shelter, dtype: int64
0    25811
1      405
Name: clothing, dtype: int64
0    25612
1      604
Name: money, dtype: int64
0    25918
1      298
Name: missing_people, dtype: int64
0    25341
1      875
Name: refugees, dtype: int64
0    25022
1     1194
Name: death, dtype: int64
0    22770
1     3446
Name: other_aid, dtype: int64
0    24511
1     1705
Name:

In [6]:

Y = Y.drop(['child_alone'], axis = 1)# child_alone has all 0

### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    '''The function will tokenize the text
    return the cleaned text'''
    #Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
     # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer# lemmatize, normalize case, and remove leading/trailing white space
    cleaned_tokens = [WordNetLemmatizer().lemmatize(w.strip()) for w in tokens]
    
    return cleaned_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
#Build pipeline
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
      
    ])



### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
#Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [9]:
# train classifier
pipeline.fit(X_train,y_train)
# predict on test data
y_pred = pipeline.predict(X_test)

In [10]:
y_train_pred = pipeline.predict(X_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
print(classification_report(y_test.values, y_pred,target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.93      0.88      5052
               request       0.80      0.42      0.55      1107
                 offer       0.00      0.00      0.00        30
           aid_related       0.77      0.56      0.65      2745
          medical_help       0.52      0.07      0.12       512
      medical_products       0.83      0.08      0.15       353
     search_and_rescue       0.57      0.07      0.13       169
              security       0.40      0.02      0.03       122
              military       0.88      0.11      0.19       216
           child_alone       0.00      0.00      0.00         0
                 water       0.82      0.25      0.38       435
                  food       0.85      0.40      0.55       742
               shelter       0.84      0.18      0.30       606
              clothing       0.77      0.09      0.17       106
                 money       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [12]:
print(classification_report(y_train.values, y_train_pred,target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.99      1.00      1.00     15042
               request       1.00      0.94      0.97      3367
                 offer       1.00      0.78      0.88        88
           aid_related       1.00      0.97      0.98      8115
          medical_help       1.00      0.85      0.92      1572
      medical_products       1.00      0.84      0.91       960
     search_and_rescue       1.00      0.81      0.90       555
              security       1.00      0.76      0.86       349
              military       1.00      0.89      0.94       644
           child_alone       0.00      0.00      0.00         0
                 water       1.00      0.93      0.96      1237
                  food       1.00      0.95      0.97      2181
               shelter       1.00      0.91      0.95      1708
              clothing       1.00      0.87      0.93       299
                 money       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
parameters ={'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 20, 25,30 ], 
              'clf__estimator__min_samples_split':[2, 5, 10]
              }
'''{'clf__estimator_min_samples_leaf': [1, 2, 4]
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4],
     yt32   
    }'''


# The gridsearch is so slow, so I choose randomized search. 
cv = RandomizedSearchCV(pipeline, param_distributions=parameters,random_state=0)
cv.fit(X_train,y_train)

RandomizedSearchCV(cv=None, error_score='raise',
          estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'vect__min_df': [1, 5], 'tfidf__use_idf': [True, False], 'clf__estimator__n_estimators': [10, 20, 25, 30], 'clf__estimator__min_samples_split': [2, 5, 10]},
          pre_dispatch='2*n_jobs', random_state=0, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
y_pred_test_cv = cv.predict(X_test)
print(classification_report(y_test.values, y_pred_test_cv, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.81      0.96      0.88      4981
               request       0.84      0.48      0.61      1105
                 offer       0.00      0.00      0.00        33
           aid_related       0.75      0.67      0.71      2657
          medical_help       0.61      0.11      0.19       513
      medical_products       0.71      0.17      0.27       296
     search_and_rescue       0.62      0.08      0.14       186
              security       0.40      0.02      0.03       125
              military       0.54      0.11      0.18       198
           child_alone       0.00      0.00      0.00         0
                 water       0.88      0.42      0.57       415
                  food       0.85      0.59      0.70       729
               shelter       0.79      0.41      0.54       553
              clothing       0.78      0.23      0.35       109
                 money       0.80      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [9]:
# Build a custom transformer which will extract the starting verb of a sentence
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class
    
    This class extract the starting verb of a sentence,
    creating a new feature for the ML classifier
    """

    def starting_verb(self, text):
        token_list = tokenize(text)
        for token in token_list:
            pos_tags = nltk.pos_tag(token)
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    # Given it is a tranformer we can return the self 
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [28]:
#Build pipeline
pipeline2 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb_transformer', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])


In [31]:
parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        
    }


In [17]:
cv_improve = RandomizedSearchCV(pipeline2, param_distributions=parameters,random_state=0)
cv_improve.fit(X_train,y_train)

RandomizedSearchCV(cv=None, error_score='raise',
          estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)), 'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0), 'features__text_pipeline__vect__max_features': (None, 5000, 10000), 'features__text_pipeline__tfidf__use_idf': (True, False)},
          pre_dispatch='2*n_jobs', random_state=0, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [19]:
y_pred_test_cv_improve = cv_improve.predict(X_test)
print(classification_report(y_test.values, y_pred_test_cv_improve, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.94      0.88      5006
               request       0.76      0.52      0.62      1139
                 offer       0.10      0.04      0.06        26
           aid_related       0.75      0.59      0.66      2691
          medical_help       0.60      0.27      0.37       533
      medical_products       0.64      0.36      0.46       302
     search_and_rescue       0.54      0.18      0.27       175
              security       0.30      0.08      0.13       137
              military       0.56      0.33      0.41       214
                 water       0.76      0.64      0.69       410
                  food       0.79      0.69      0.73       695
               shelter       0.74      0.54      0.62       554
              clothing       0.64      0.42      0.51        81
                 money       0.61      0.32      0.42       155
        missing_people       0.48      

### 9. Export your model as a pickle file

In [37]:
m = pickle.dumps(cv,open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.